In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

## Load Data

In [2]:
normalization = "min_max_normalization" # "standardization"

In [3]:
###
#  load data: targets and features
###
if normalization == 'min_max_normalization':
    file = h5py.File("targets_and_normalized_features.jld2", "r")
else:
    file = h5py.File("targets_and_standardized_features.jld2", "r")

# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

In [4]:
###
#  load data: initializing COFs
###
init_cof_ids_file = pickle.load(open('search_results/{}/initializing_cof_ids.pkl'.format(normalization), 'rb'))
init_cof_ids = init_cof_ids_file['init_cof_ids']

# total number of BO searches to run = number of initializing sets
nb_runs = len(init_cof_ids)

###
#  print info
###
print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([5.6399e+01, 9.2784e-01, 6.3570e+03, 1.6107e+03, 1.8182e-01, 2.5000e-01,
        6.6667e-01, 5.0000e-01, 2.9520e-02, 3.3333e-01, 1.4286e-01, 6.6667e-02,
        2.8571e-01, 2.3810e-02], dtype=torch.float64)
min:
 tensor([3.5094e+00, 1.6356e-01, 1.9966e+03, 1.0272e+02, 0.0000e+00, 0.0000e+00,
        3.2500e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], dtype=torch.float64)
width:
 tensor([5.2889e+01, 7.6428e-01, 4.3604e+03, 1.5080e+03, 1.8182e-01, 2.5000e-01,
        3.4167e-01, 5.0000e-01, 2.9520e-02, 3.3333e-01, 1.4286e-01, 6.6667e-02,
        2.8571e-01, 2.3810e-02], dtype=torch.float64)


In [5]:
X_unsqueezed = X.unsqueeze(1)

## Helper Functions

#### Analysis

In [6]:
# accumulated cost lags the index of the cost_acquired (iteration COF is identified) by 1
def accumulated_cost(cost_acquired):
    nb_iters = len(cost_acquired)
    accumulated_cost = np.zeros(nb_iters)
    accumulated_cost[0] = cost_acquired[0]
    for i in range(1, len(cost_acquired)):
        accumulated_cost[i] = accumulated_cost[i-1] + cost_acquired[i]
    return accumulated_cost

In [7]:
def get_y_maxes_acquired(y_acquired):
    nb_iters = len(y_acquired)
    return [max(y_acquired[:i+1]) for i in range(nb_iters)]      

#### Construct Initial Inputs

In [8]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [9]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1))
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [10]:
def run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=False):
    assert nb_iterations > len(initializing_COFs)
    ###
    #  initialize system
    ###
    # select initial COFs for training data
    ids_acquired = initializing_COFs
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(len(initializing_COFs), nb_iterations):
        # construct training data (perform experiments)
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # identify COF with highest acquisition value currently *not* in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        if verbose:
            print("BO iteration ", i)
            print("\tacquired COF ", id_max_acq)
            print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [11]:
nb_iterations = 150
nb_COFs_initialization = 3

In [12]:
for i, initializing_COFs in enumerate(init_cof_ids): 
    # check the length of each initializing set
    assert len(initializing_COFs) == nb_COFs_initialization
    print("run #: {}".format(i))
    
    # start timer for BO run
    start_time    = time.time()
    ###
    #  run BO search
    ###
    ids_acquired  = run_Bayesian_optimization(nb_iterations, initializing_COFs)
    
    ###
    #  post-run analysis
    ###
    elapsed_time  = time.time() - start_time
    y_acquired    = y[ids_acquired]
    y_maxes_acq   = get_y_maxes_acquired(y_acquired.detach().numpy())
    cost_acquired = build_cost(ids_acquired)
    acc_cost      = accumulated_cost(cost_acquired.detach().numpy())
        
    BO_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
    print("iteration we acquire top COF = ", BO_iter_top_cof_acquired.item())
    
    top_cof_acc_cost = sum(cost_acquired[:BO_iter_top_cof_acquired]).item() 
    print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]")
    
    ###
    #  Store SFBO results
    ###
    sfbo_res = dict({'ids_acquired': ids_acquired,
                     'y_acquired': y_acquired.detach().numpy(),
                     'y_max_acquired': y_maxes_acq,
                     'cost_acquired': cost_acquired.detach().numpy(),
                     'accumulated_cost': acc_cost / 60,
                     'nb_COFs_initialization': nb_COFs_initialization,
                     'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired.item(),
                     'elapsed_time (min)':  elapsed_time / 60
                    })

    with open('search_results/{}/sfbo_results/sfbo_results_run_{}.pkl'.format(normalization, i), 'wb') as file:
        pickle.dump(sfbo_res, file)

run #: 0
Iter 10/100: 2.5307720700453147
Iter 20/100: 1.5976922034309282
Iter 30/100: 1.5336722936520173
Iter 40/100: 1.5099266078913303
Iter 50/100: 1.4668760781869867
Iter 60/100: 1.463951806283072
Iter 10/100: 2.292175249396065
Iter 20/100: 1.586213848781102
Iter 30/100: 1.5277763282048402
Iter 40/100: 1.496472490674787
Iter 50/100: 1.4631640822005751
Iter 60/100: 1.4607431298082354
Iter 70/100: 1.4604264289151632
Iter 80/100: 1.4582039854011923
Iter 10/100: 2.1489509890939735
Iter 20/100: 1.5782942677669698
Iter 30/100: 1.5189735767833439
Iter 40/100: 1.4832453036058006
Iter 50/100: 1.4579556806535785
Iter 60/100: 1.4563432681700708
Iter 70/100: 1.4555677208384283
Iter 80/100: 1.4538109359842002
Iter 10/100: 2.0534795690477745
Iter 20/100: 1.5711002610931193
Iter 30/100: 1.5080101059879514
Iter 40/100: 1.4767109558028062
Iter 50/100: 1.4536812678453224
Iter 60/100: 1.4514249000585357
Iter 10/100: 1.9851170596855157
Iter 20/100: 1.5627728093406443
Iter 30/100: 1.4989776444991436
Ite

Iter 10/100: 1.5758356880483515
Iter 20/100: 1.461592286320934
Iter 30/100: 1.4427001781517457
Iter 10/100: 1.5749580515989576
Iter 20/100: 1.4612789832312323
Iter 30/100: 1.4424001676322622
Iter 10/100: 1.5741228574417343
Iter 20/100: 1.4610029869000698
Iter 30/100: 1.4421239330406443
Iter 10/100: 1.5733060534690617
Iter 20/100: 1.460722147107036
Iter 30/100: 1.4418479178999624
Iter 10/100: 1.5725136410236964
Iter 20/100: 1.460448639124074
Iter 30/100: 1.4415792314764615
Iter 10/100: 1.5717453905299623
Iter 20/100: 1.460184878299848
Iter 30/100: 1.4413182425143947
Iter 10/100: 1.570999455911497
Iter 20/100: 1.459928061254012
Iter 30/100: 1.441064074170912
Iter 10/100: 1.5702739426233077
Iter 20/100: 1.459675002869047
Iter 30/100: 1.4408156223620103
Iter 10/100: 1.5695714707255146
Iter 20/100: 1.4594357645462017
Iter 30/100: 1.4405756123121907
Iter 10/100: 1.5688876653101091
Iter 20/100: 1.4592001048245677
Iter 30/100: 1.440340899127163
Iter 10/100: 1.568222835665034
Iter 20/100: 1.458

Iter 20/100: 1.4506002613540157
Iter 30/100: 1.4314651597207373
Iter 10/100: 1.5438301245847104
Iter 20/100: 1.4505469784704415
Iter 30/100: 1.431408451062628
iteration we acquire top COF =  119
accumulated cost up to observation of top COF =  33201.11231125593  [min]
run #: 1
Iter 10/100: 2.530771623809402
Iter 20/100: 1.5976920793168514
Iter 30/100: 1.5336718522650357
Iter 40/100: 1.5099265668793855
Iter 50/100: 1.4668759232376107
Iter 60/100: 1.463951870588361
Iter 10/100: 2.292174692842523
Iter 20/100: 1.5862137328300612
Iter 30/100: 1.5277756963227025
Iter 40/100: 1.4964727573656018
Iter 50/100: 1.463164045810668
Iter 60/100: 1.460743124838842
Iter 70/100: 1.4604264836071092
Iter 80/100: 1.458203959776168
Iter 10/100: 2.1489893007474494
Iter 20/100: 1.5782973587153115
Iter 30/100: 1.5189801117663877
Iter 40/100: 1.483249303585485
Iter 50/100: 1.45795849789981
Iter 60/100: 1.4563429540963502
Iter 70/100: 1.4555677837645442
Iter 80/100: 1.4538110300059128
Iter 10/100: 2.053479937341

Iter 30/100: 1.4439592888552013
Iter 10/100: 1.5786229646280836
Iter 20/100: 1.4625516119348039
Iter 30/100: 1.4436302396142005
Iter 10/100: 1.5776504898291246
Iter 20/100: 1.4622049237927583
Iter 30/100: 1.4432998149056209
Iter 10/100: 1.5767225767666582
Iter 20/100: 1.4618860438886394
Iter 30/100: 1.4429906038329166
Iter 10/100: 1.5758250916864966
Iter 20/100: 1.4615779158943787
Iter 30/100: 1.4426908722683927
Iter 10/100: 1.5749563583084685
Iter 20/100: 1.461279879121157
Iter 30/100: 1.4423999841708224
Iter 10/100: 1.574115373986847
Iter 20/100: 1.4609919575187977
Iter 30/100: 1.4421179104776727
Iter 10/100: 1.5733009614155786
Iter 20/100: 1.4607140752461112
Iter 30/100: 1.441844264052782
Iter 10/100: 1.5725097093201328
Iter 20/100: 1.4604426565156705
Iter 30/100: 1.441576582733281
Iter 10/100: 1.5717433397804719
Iter 20/100: 1.4601814412947325
Iter 30/100: 1.4413170806886972
Iter 10/100: 1.570998541960968
Iter 20/100: 1.4599264779710754
Iter 30/100: 1.4410636134959889
Iter 10/100: 

Iter 20/100: 1.450750263963739
Iter 30/100: 1.431631062140959
Iter 10/100: 1.544289015376456
Iter 20/100: 1.4506948819539045
Iter 30/100: 1.4315721989608121
Iter 10/100: 1.5441305565641503
Iter 20/100: 1.4506399809925061
Iter 30/100: 1.4315138491047463
Iter 10/100: 1.5439746909266476
Iter 20/100: 1.4505863191729198
Iter 30/100: 1.4314567530936568
Iter 10/100: 1.5438212209901094
Iter 20/100: 1.4505337336323478
Iter 30/100: 1.4314007274878808
iteration we acquire top COF =  120
accumulated cost up to observation of top COF =  33442.41581608851  [min]
run #: 2
Iter 10/100: 2.5307326354667157
Iter 20/100: 1.5976893495295865
Iter 30/100: 1.5336665184179006
Iter 40/100: 1.5099289953192965
Iter 50/100: 1.4668743090037377
Iter 60/100: 1.4639513553372119
Iter 10/100: 2.29213815697132
Iter 20/100: 1.5861811162993595
Iter 30/100: 1.5277677996203443
Iter 40/100: 1.4964705459433671
Iter 50/100: 1.463163868289834
Iter 60/100: 1.4607439649871885
Iter 70/100: 1.4604261930982616
Iter 80/100: 1.45820387

...

Iter 10/100: 1.5490936737899896
Iter 20/100: 1.4523536932983028
Iter 30/100: 1.4333256442763653
Iter 10/100: 1.5488638300506676
Iter 20/100: 1.4522744019225882
Iter 30/100: 1.4332423109458234
Iter 10/100: 1.5486377369562148
Iter 20/100: 1.4521964170777908
Iter 30/100: 1.433160304156859
Iter 10/100: 1.5484131086243778
Iter 20/100: 1.4521179545246485
Iter 30/100: 1.4330773697178072
Iter 10/100: 1.5481938914155702
Iter 20/100: 1.4520421728128772
Iter 30/100: 1.4329975492591096
Iter 10/100: 1.5479786265021902
Iter 20/100: 1.4519679469966684
Iter 30/100: 1.4329194227958932
Iter 10/100: 1.54776653157192
Iter 20/100: 1.4518947298407268
Iter 30/100: 1.432842272724734
Iter 10/100: 1.5475577246866816
Iter 20/100: 1.4518226385944504
Iter 30/100: 1.4327662657178455
Iter 10/100: 1.5473523034039145
Iter 20/100: 1.451751788311174
Iter 30/100: 1.4326915534836762
Iter 10/100: 1.5471497770436888
Iter 20/100: 1.4516818129130828
Iter 30/100: 1.4326176849449106
Iter 10/100: 1.546950537970491
Iter 20/100: 1

Iter 10/100: 1.6306472865814508
Iter 20/100: 1.4803072797954027
Iter 30/100: 1.458744788323575
Iter 10/100: 1.627221687267625
Iter 20/100: 1.4791486056074992
Iter 30/100: 1.4579085156937825
Iter 10/100: 1.6240041295388816
Iter 20/100: 1.478060359843991
Iter 30/100: 1.4570994094036702
Iter 10/100: 1.6209246255847392
Iter 20/100: 1.4770156555364595
Iter 30/100: 1.4562973430757897
Iter 10/100: 1.6181193582721956
Iter 20/100: 1.4760623685798109
Iter 30/100: 1.455561975240288
Iter 10/100: 1.6154225060249514
Iter 20/100: 1.4751465266985484
Iter 30/100: 1.4548338325238446
Iter 10/100: 1.6128714423393262
Iter 20/100: 1.4742783100580206
Iter 30/100: 1.454131699753496
Iter 10/100: 1.6104540985148186
Iter 20/100: 1.4734545818202947
Iter 30/100: 1.4534548663677878
Iter 10/100: 1.6081602356514986
Iter 20/100: 1.472669830832844
Iter 30/100: 1.4528020318032657
Iter 10/100: 1.6059830530105281
Iter 20/100: 1.4719300565568019
Iter 30/100: 1.4521745133246977
Iter 10/100: 1.6039108870087215
Iter 20/100: 1

Iter 20/100: 1.4526835972445458
Iter 30/100: 1.433671522492659
Iter 10/100: 1.549805861411868
Iter 20/100: 1.452599163695288
Iter 30/100: 1.4335831563100636
Iter 10/100: 1.5495643390099085
Iter 20/100: 1.4525158462835714
Iter 30/100: 1.4334957809440079
Iter 10/100: 1.5493268689100548
Iter 20/100: 1.452433949235631
Iter 30/100: 1.433409846393959
Iter 10/100: 1.5490932392551569
Iter 20/100: 1.4523533535417166
Iter 30/100: 1.433325207108489
Iter 10/100: 1.548863740920442
Iter 20/100: 1.452274329251464
Iter 30/100: 1.4332422198477637
Iter 10/100: 1.548637356296109
Iter 20/100: 1.4521961012369689
Iter 30/100: 1.4331599129017458
Iter 10/100: 1.5484150553945195
Iter 20/100: 1.452119494884178
Iter 30/100: 1.4330793351914102
Iter 10/100: 1.5481939581555542
Iter 20/100: 1.4520422220416165
Iter 30/100: 1.4329976151143828
Iter 10/100: 1.5479784763985867
Iter 20/100: 1.4519678326544212
Iter 30/100: 1.4329192734886398
Iter 10/100: 1.547766612059742
Iter 20/100: 1.4518947930552666
Iter 30/100: 1.4328

Iter 10/100: 1.6423921453703583
Iter 20/100: 1.4842598465598866
Iter 30/100: 1.461401197404135
Iter 10/100: 1.6382070434163267
Iter 20/100: 1.4828545765601833
Iter 30/100: 1.4604932299184736
Iter 10/100: 1.6342982190840225
Iter 20/100: 1.4815323676822238
Iter 30/100: 1.4596046788747186
Iter 10/100: 1.6306465415768014
Iter 20/100: 1.4803045765649017
Iter 30/100: 1.45874426425959
Iter 10/100: 1.627222032315403
Iter 20/100: 1.479149943515903
Iter 30/100: 1.4579087241138096
Iter 10/100: 1.6240029166429213
Iter 20/100: 1.478058361815246
Iter 30/100: 1.4570989592992958
Iter 10/100: 1.6209754202820736
Iter 20/100: 1.4770333036271106
Iter 30/100: 1.4563172588381872
Iter 10/100: 1.6181197168648156
Iter 20/100: 1.4760634674631523
Iter 30/100: 1.4555621811388977
Iter 10/100: 1.615422761137236
Iter 20/100: 1.475146850877702
Iter 30/100: 1.4548339086567716
Iter 10/100: 1.612871059249085
Iter 20/100: 1.4742771410282316
Iter 30/100: 1.4541314673033556
Iter 10/100: 1.6104542995344728
Iter 20/100: 1.47

Iter 20/100: 1.4529505770657447
Iter 30/100: 1.4339521531310055
Iter 10/100: 1.5505582505314044
Iter 20/100: 1.4528596374045772
Iter 30/100: 1.4338565847083444
Iter 10/100: 1.5503038942933562
Iter 20/100: 1.4527718822539224
Iter 30/100: 1.4337646774075519
Iter 10/100: 1.5500542486763647
Iter 20/100: 1.4526859253067226
Iter 30/100: 1.4336746579121848
Iter 10/100: 1.549808641949789
Iter 20/100: 1.4526012920991167
Iter 30/100: 1.4335859244521885
Iter 10/100: 1.5495674273252003
Iter 20/100: 1.4525183325162212
Iter 30/100: 1.4334989139566612
Iter 10/100: 1.5493273126531464
Iter 20/100: 1.4524342849416318
Iter 30/100: 1.43341028692138
Iter 10/100: 1.5490937736692327
Iter 20/100: 1.4523537729996416
Iter 30/100: 1.4333257455159474
Iter 10/100: 1.548863652322625
Iter 20/100: 1.4522742575250718
Iter 30/100: 1.433242129511697
Iter 10/100: 1.548637432282802
Iter 20/100: 1.4521961635596956
Iter 30/100: 1.4331599908019181
Iter 10/100: 1.5484130154530504
Iter 20/100: 1.452117886535821
Iter 30/100: 1.

# Random Search

In [13]:
nb_runs = 1000

rs_res = dict()
rs_res['ids_acquired']     = []
rs_res['cost_acquired']    = []
rs_res['accumulated_cost'] = []

for r in range(nb_runs):
    rs_ids_acquired     = np.random.choice(range(nb_COFs), replace=False, size=nb_iterations)
    rs_cost_acquired    = build_cost(rs_ids_acquired)
    rs_accumulated_cost = accumulated_cost(rs_cost_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)
    rs_res['accumulated_cost'].append(rs_accumulated_cost / 60)

In [14]:
def test_rs(rs_res):
    for a in np.random.choice(nb_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_iterations
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [15]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_runs)])
        assert np.size(y_maxes) == nb_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [16]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired'],
                             'accumulated_cost': rs_res['accumulated_cost']
                             })

with open('search_results/{}/random_search_results.pkl'.format(normalization), 'wb') as file:
    pickle.dump(random_search_results, file)